In [ ]:
batch_size = 10
num_characters = 21
seq_length = 238
original_dim = num_characers * seq_length
latent_dim = 20
intermediate_dim = 50
epochs = 100
epsilon_std = 1.0

x = Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim, activation='elu')(x)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(intermediate_dim, activation='elu')
decoder_mean = Dense(original_dim, activation='elu')
h_decoded = decoder_h(z)
x_decoded = decoder_mean(h_decoded)
x_reshape = Reshape((seq_length, num_characters))(x_decoded)
x_decoded_mean = Dense(num_characters, activation='softmax')(x_reshape)

# placeholder loss
def zero_loss(y_true, y_pred):
    return K.zeros_like(y_pred)

# Custom loss layer
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x, x_decoded_mean):
        xent_loss = K.sum(K.categorical_crossentropy(x, x_decoded_mean), axis = -1)
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.sum(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        loss = self.vae_loss(x, x_decoded_mean)
        self.add_loss(loss, inputs=inputs)
        # we don't use this output, but it has to have the correct shape:
        return K.ones_like(x)

loss_layer = CustomVariationalLayer()([x, x_decoded_mean])
vae = Model(x, [loss_layer])
vae.compile(optimizer='rmsprop', loss=[zero_loss])

#checkpoint
cp = [callbacks.ModelCheckpoint(filepath="/home/ubuntu/pynb/model.h5", verbose=1, save_best_only=True)]

#train
vae.fit(train, train.reshape(-1, seq_length, num_characters),
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(test, test), callbacks=cp)

# build a model to project inputs on the latent space
encoder = Model(x, z_mean)

# build a generator that can sample from the learned distribution
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import keras
from keras import backend as K
from keras.layers import Lambda, Input, Dense, Reshape
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy, categorical_crossentropy
from keras.utils import plot_model

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os


# reparameterization trick
# instead of sampling from Q(z|X), sample epsilon = N(0,I)
# z = z_mean + sqrt(var) * epsilon
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    print(z_mean)
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [2]:
from util import load_gfp_data, count_substring_mismatch, get_all_amino_acids, get_wild_type_amino_acid_sequence
from util import one_hot_encode
train_size = 1100
num_amino_acids = 21
seq_length = 238
input_length = num_amino_acids * seq_length
X_train, X_test, y_train, y_test = load_gfp_data("../../data/gfp_amino_acid_")
X_train = one_hot_encode(X_train[0:train_size], get_all_amino_acids())
print(X_train.shape)

(1100, 4998)


In [3]:
# network parameters
input_shape = (input_length, )
intermediate_dim = 50
batch_size = 10
latent_dim = 20
epochs = 100

# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
expected = Reshape((seq_length, num_amino_acids))(inputs)
x = Dense(intermediate_dim, activation='elu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()
plot_model(encoder, to_file='./logs/vae_mlp_encoder.png', show_shapes=True)

# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='elu')(latent_inputs)
x = Dense(input_length)(x)        
x = Reshape((seq_length, num_amino_acids), name='d4')(x)
outputs = Dense(num_amino_acids, activation='softmax')(x)
decoder = Model(latent_inputs, outputs, name='decoder')
print(decoder.summary())
plot_model(decoder, to_file='./logs/vae_mlp_decoder.png', show_shapes=True)

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

Tensor("z_mean/BiasAdd:0", shape=(?, 20), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 4998)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 50)           249950      encoder_input[0][0]              
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 20)           1020        dense_1[0][0]                    
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 20)           1020        dense_1[0][0]                    
____________________________________________________

In [8]:
reconstruction_loss = K.sum(K.categorical_crossentropy(expected, outputs), axis=-1)
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
print(vae.summary())
plot_model(vae,
           to_file='vae_mlp.png',
           show_shapes=True)
load_weights=False
if load_weights:
    vae.load_weights(args.weights)
else:
    # train the autoencoder
    vae.fit(X_train, 
            epochs=epochs,
            batch_size=batch_size,
            validation_split = 0.1)
    vae.save_weights('./models/vae_mlp_mnist.h5')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 4998)              0         
_________________________________________________________________
encoder (Model)              [(None, 20), (None, 20),  251990    
_________________________________________________________________
decoder (Model)              (None, 238, 21)           256410    
Total params: 508,400
Trainable params: 508,400
Non-trainable params: 0
_________________________________________________________________
None
Train on 990 samples, validate on 110 samples
Epoch 1/100
990/990 [==============================] - 4s 4ms/step - loss: 359.7659 - val_loss: 987.6673
Epoch 2/100
990/990 [==============================] - 3s 3ms/step - loss: 343.8110 - val_loss: 971.0567
Epoch 3/100
990/990 [==============================] - 2s 3ms/step - loss: 339.0501 - val_loss: 954.6153
Epoch 4/100
990/990 [=============

990/990 [==============================] - 2s 2ms/step - loss: 262.2282 - val_loss: 1137.5557
Epoch 71/100
990/990 [==============================] - 2s 2ms/step - loss: 261.3621 - val_loss: 1130.2733
Epoch 72/100
990/990 [==============================] - 2s 2ms/step - loss: 262.8988 - val_loss: 1166.3161
Epoch 73/100
990/990 [==============================] - 2s 2ms/step - loss: 260.1511 - val_loss: 1129.6734
Epoch 74/100
990/990 [==============================] - ETA: 0s - loss: 261.300 - 2s 2ms/step - loss: 262.2736 - val_loss: 1134.3404
Epoch 75/100
990/990 [==============================] - 2s 2ms/step - loss: 261.2003 - val_loss: 1152.3390
Epoch 76/100
990/990 [==============================] - 2s 2ms/step - loss: 264.7140 - val_loss: 1156.6776
Epoch 77/100
990/990 [==============================] - 3s 3ms/step - loss: 260.4358 - val_loss: 1171.0082
Epoch 78/100
990/990 [==============================] - 2s 3ms/step - loss: 257.8718 - val_loss: 1212.9316
Epoch 79/100
990/990 [==